# Goal 

Ways to apply transformations to columns (all and subset) in PySpark. 

### Purpose
Fundamental understanding and implementation

In [55]:
import math;

In [28]:
import findspark
findspark.init()

import pyspark;
from pyspark.ml.feature import *
from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext, SparkSession
import pyspark.sql.functions as f
import functools

In [2]:
spark = SparkSession.builder.master('local').appName('playground').config('spark.driver.memory', '5G').getOrCreate()
spark.builder.config('spark.executor.memory', '16G')
spark.builder.config("spark.executor.cores", "4")

## 1. Using Pythons functools REDUCE

In [75]:
# Delibrately simple to understand transformations quickly

raw_df = spark.createDataFrame(
    [
        (10, 100),
        (10000, 1000000)
    ],
    ["Column1", "Column2"] # column names
)

raw_df.show()

+-------+-------+
|Column1|Column2|
+-------+-------+
|     10|    100|
|  10000|1000000|
+-------+-------+



In [77]:
using_reduce_df = functools.reduce(
    
    # 1. function 
    lambda df, column_name: df.withColumn(column_name, #col name
                                              
                                              # Apply transformation in Spark here
                                              f.log10( 
                                                  
                                               f.col(column_name) # retrieve actual column object values
                                           
                                            )# T end
                                          ),
    # 2. thing you want to iterate over (iterable)
    raw_df.columns, 
    
    # 3. raw sequence data
    raw_df
)

using_reduce_df.show()

+-------+-------+
|Column1|Column2|
+-------+-------+
|    1.0|    2.0|
|    4.0|    6.0|
+-------+-------+



In [78]:
spark.stop()